In [1]:
import os
os.chdir("..")
import sys
sys.path.append(os.getcwd())
print(os.getcwd())

/home/development/SPOT/backend-zoning


In [2]:
import json

from jeffutils.utils import reimport
reimport("""

""", globals())
import modules.data_manager as dm

import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from jeffutils.analytics import set_np_pd_display_params
from jeffutils.utils import reimport
set_np_pd_display_params(np, pd)

reimport("""
import modules.data_manager as dm
""", globals())
import modules.data_manager as dm

In [3]:
video_name = "2025-02-14-17-25-00_000004-1-JFSBP1_center_east"
date_str, cam_label = dm.get_date_str_cam_label(video_name)
print(date_str, cam_label)

tracking_csv_path = dm.get_jfsb_tracking_csv_path(video_name)
tracking_df = pd.read_csv(tracking_csv_path)

motion_csv_path = dm.get_jfsb_motion_csv_path(video_name)
motion_df = pd.read_csv(motion_csv_path)

info_csv_path = dm.get_jfsb_info_csv_path()
info_df = pd.read_csv(info_csv_path)

video_path = dm.get_jfsb_video_path(video_name)



2025-02-14-17-25-00 000004-1-JFSBP1_center_east


# display number of enter/exit videos per region

In [4]:
for action, action_frame in info_df.groupby('which_action'):
    print(action)
    display(action_frame['which_parking_zone'].value_counts())

enter


which_parking_zone
JFSBP1_center_west-region_1-north_west    51
JFSBP2_east_west-region_0-south_east      32
JFSBP2_west_east-region_3-south_west      26
JFSBP1_east_west-region_0-south_east      19
JFSBP2_center_west-region_0-south_west    19
JFSBP2_center_east-region_0-north_east    18
JFSBP1_center_east-region_0-north_east    16
JFSBP2_center_east-region_1-south_east    16
JFSBP2_east_west-region_1-north_east      10
JFSBP1_center_east-region_1-south_east     9
JFSBP2_west_east-region_0-north_west       8
JFSBP1_east_west-region_1-north_east       7
JFSBP1_west_east-region_1-south_west       7
JFSBP1_center_west-region_0-south_west     5
JFSBP1_west_east-region_0-north_west       4
JFSBP2_center_west-region_1-north_west     3
Name: count, dtype: int64

exit


which_parking_zone
JFSBP2_center_east-region_0-north_east    22
JFSBP2_east_west-region_0-south_east      21
JFSBP2_west_east-region_3-south_west      21
JFSBP1_east_west-region_0-south_east      19
JFSBP1_center_east-region_0-north_east    16
JFSBP1_east_west-region_1-north_east      11
JFSBP1_center_west-region_1-north_west    10
JFSBP2_center_east-region_1-south_east     9
JFSBP2_east_west-region_1-north_east       9
JFSBP1_west_east-region_1-south_west       8
JFSBP2_center_west-region_0-south_west     8
JFSBP2_center_west-region_1-north_west     5
JFSBP2_west_east-region_0-north_west       3
JFSBP1_center_east-region_1-south_east     2
JFSBP1_center_west-region_0-south_west     2
JFSBP1_west_east-region_0-north_west       1
Name: count, dtype: int64

# test the data_manager methods

In [10]:
zoneID = "JFSBP1_center_west-region_0-south_west"

cam_label, region_id, zone_name = dm.parse_zoneID(zoneID)
print(cam_label, region_id, zone_name)

zone_settings = dm.get_zone_settings(zoneID)
print(json.dumps(zone_settings, indent=2))

cam_resolution = dm.get_camera_resolution(cam_label)
print(json.dumps(cam_resolution, indent=2))

settings = dm.get_settings(cam_label)
print(json.dumps(settings, indent=2))

000004-1-JFSBP1_center_west region_0 south_west
{
  "zone_name": "south_west",
  "direction": "LEFT",
  "points": [
    [
      0.0,
      0.0
    ],
    [
      0.46836,
      0.0
    ],
    [
      0.46759,
      0.02675
    ],
    [
      0.37269,
      0.09053
    ],
    [
      0.29282,
      0.16307
    ],
    [
      0.08063,
      0.40484
    ],
    [
      0.0,
      0.53447
    ]
  ]
}
{
  "height": 1944,
  "width": 2592
}
{
  "cam_type": "overhead-lpr",
  "camera_label": "000004-1-JFSBP1_center_west",
  "lot": "1",
  "organizationID": "000004",
  "picture_path": "settings/pictures/000004-1-JFSBP1_center_west.png",
  "resolution": {
    "height": 1944,
    "width": 2592
  },
  "rtsp_link": "rtsp://study:w@tchTh3carzcra5h!@10.11.60.80:654/00000003-0000-babe-0000-b8a44fb19729/live?redirected=True",
  "passoff_regions": {
    "JFSBP1_center_west-region_2": {
      "direction": "UP",
      "points": [
        [
          0.0,
          0.71451
        ],
        [
          1.0,
